Paquetes

In [1]:
import requests
import configparser
import pandas as pd
import numpy as np
import json
import time as systime
import ast

Parametros de la cuenta (Demo)

In [2]:
token = 'feda87f1f9a79663042fcd5ee82ab17c-22f342d828f6cfd14b6004a04fdb6283'
account = '101-011-11815575-001'
username = 'kgua0311'

Parametros de conexion a la API

In [3]:
header = {"Content-Type": "application/json", "Authorization": "Bearer " + token}
pricing_url = 'https://api-fxpractice.oanda.com/v3/accounts/' + account + "/pricing"
order_url = 'https://api-fxpractice.oanda.com/v3/accounts/' + account + "/orders"
pricing_stream = 'https://stream-fxpractice.oanda.com/v3/accounts/' + account + "/pricing/stream"
instruments_url = "https://api-fxpractice.oanda.com/v3/accounts/" + account + "/instruments"

Instrumentos

In [4]:
Tickers = ['USD_CNH', 'MBTC_USD', 'XAU_CAD', 'SGD_HKD', 'SPX500_USD', 'EUR_HUF', 'AUD_NZD', 'CAD_JPY',
           'EU50_EUR', 'USB10Y_USD', 'ZAR_JPY', 'EUR_AUD', 'DE30_EUR', 'XCU_USD', 'XAG_GBP', 'HKD_JPY',
           'XAG_JPY', 'XAG_USD', 'USD_NOK', 'CAD_SGD', 'USD_CZK', 'GBP_NZD', 'XAG_NZD', 'FR40_EUR', 
           'EUR_DKK', 'USD_CAD', 'AUD_JPY', 'USD_HUF', 'EUR_CZK', 'AUD_HKD', 'CHF_ZAR', 'SUGAR_USD',
           'EUR_HKD', 'AU200_AUD', 'NZD_USD', 'AUD_USD', 'SGD_JPY', 'AUD_CAD', 'WTICO_USD', 'DE10YB_EUR',
           'CORN_USD', 'AUD_CHF', 'CN50_USD', 'NL25_EUR', 'BTC_USD', 'TWIX_USD', 'EUR_CHF', 'USB02Y_USD',
           'SG30_SGD', 'EUR_NOK', 'AUD_SGD', 'USD_CHF', 'TRY_JPY', 'USD_THB', 'GBP_CHF', 'USD_PLN', 'GBP_AUD',
           'EUR_SGD', 'CHF_JPY', 'USD_INR', 'EUR_SEK', 'USD_SGD', 'XAU_NZD', 'XAU_GBP', 'USD_DKK', 'XPT_USD',
           'XAG_CHF', 'NZD_SGD', 'XAU_XAG', 'XAG_EUR', 'UK10YB_GBP', 'SOYBN_USD', 'GBP_ZAR', 'USD_HKD',
           'XAU_USD', 'GBP_PLN', 'USD_JPY', 'EUR_TRY', 'NAS100_USD', 'EUR_ZAR', 'XAU_HKD', 'NZD_HKD',
           'NZD_CHF', 'XAU_CHF', 'GBP_CAD', 'USD_SAR', 'XAU_JPY', 'NZD_CAD', 'EUR_JPY', 'NZD_JPY', 'EUR_NZD',
           'WHEAT_USD', 'XAU_AUD', 'US30_USD', 'XAU_EUR', 'UK100_GBP', 'EUR_GBP', 'USD_SEK', 'GBP_SGD',
           'US2000_USD', 'USB05Y_USD', 'USD_ZAR', 'NATGAS_USD', 'CHF_HKD', 'XPD_USD', 'GBP_HKD', 'EUR_PLN',
           'BCO_USD', 'XAG_AUD', 'JP225_USD', 'XAG_HKD', 'EUR_USD', 'XAG_SGD', 'XAU_SGD', 'IN50_USD',
           'USD_TRY', 'GBP_JPY', 'EUR_CAD', 'HK33_HKD', 'XAG_CAD', 'USD_MXN', 'GBP_USD', 'SGD_CHF', 
           'CAD_CHF', 'CAD_HKD', 'USB30Y_USD']

# Control para determinar si hacer stream de un solo activo o varios
multiple = True

if multiple:
    Tickers_stream = ','.join(Tickers)
else:
    Tickers_stream = 'USD_JPY' # Tomar uno en especifico

In [57]:
def get_current_price(epic_id):
    pars = {"instruments": epic_id}
    session = requests.Session()  # create a session
    ob_req = requests.Request("GET", pricing_url, headers=header, params=pars)
    ob_pre = ob_req.prepare()
    resp = session.send(ob_pre)
    session.close()
    if resp.ok:  # If this is "OK" means that the sending and receiving was succesful
        # Extracting information from the json output
        dict_response = resp.json()
        res = {'Instrument': epic_id}
        res['values'] = {}
        res['values']['BID'] = dict_response["prices"][0]["bids"][0]["price"]
        res['values']['OFFER'] = dict_response["prices"][0]["asks"][0]["price"]
    return res

In [62]:
def placeOrder(epic_id, units = None, order_type = 'LIMIT', price = None):
    """
    Function to place orders in OANDA.
    :param epic_id: Ticker of the instrument
    :param order_type: one of the following as strings
                - MARKET
                - LIMIT
    :param units: integer of the number of units to buy, negative if selling
    :param price: target price, only for LIMIT, STOP_LOSS, TAKE_PROFIT orders
    """
    
    if units > 0:
        price = float(get_current_price('EUR_USD')['values']['OFFER'])
    else:
        price = float(get_current_price('EUR_USD')['values']['BID'])

    if order_type is None:
        print('No type order given. Please specify the order type.')
        return None

    if order_type == "MARKET":
        params = {"order": {"timeInForce": "GTC",
                            "instrument": epic_id,
                            "units": str(units),
                            "type": order_type,
                            "positionFill": "DEFAULT"}}

    if order_type == "LIMIT":
        params = {"order": {"price": str(price),
                            "timeInForce": "GTC",
                            "instrument": epic_id,
                            "units": str(units),
                            "type": order_type,
                            "positionFill": "DEFAULT"}}

    params = json.dumps(params)
    session = requests.Session()
    req = requests.Request("POST", order_url, headers = header, data = params)
    pre = req.prepare()
    resp = session.send(pre)
    session.close()
    print(resp.reason)
    ins = resp.json()
    print(ins)

In [63]:
def get_signal(Hist_Data, power = 0.75, Threshold_ranges = [-0.2, 0.5]):
    time_window = Hist_Data.shape[0] # It identifies the time window for past data you're using
    n = time_window**power # this is one of the parameters of the algoritm
    
    High_shift = Hist_Data['high'].shift(1) # For the current candel, I consider the last 9 candels for calculations in the High
    Low_shift = Hist_Data['low'].shift(1) # Same for the Low
    TR1 = High_shift-Low_shift # Calculate the first value of reference as an array
    TR2 = np.abs(High_shift - Hist_Data['close']) # Calculate the second value of reference as an array
    TR3 = np.abs(Low_shift-Hist_Data['close']) # Calculate the third value of reference as an array
    TR = pd.concat([TR1,TR2,TR3], axis = 1) # join them into a dataframe
    TR = [max(TR.iloc[i,:]) for i in range(TR.shape[0])] # For each row, take the maximum value
    Avr_TR = np.nanmean(TR) # calculate the mean of TR
    High_Range = Hist_Data['high'] - Hist_Data['low'].min() # Get the difference between the current High and the past min Low
    Low_Range = Hist_Data['low'] - Hist_Data['high'].max() # Get the difference between the current Low and the past max High
    NHR = High_Range/n/Avr_TR
    NHR = NHR.values[-1] # Extracts the current value of the indicator for the High
    NLR = Low_Range/n/Avr_TR
    NLR = NLR.values[-1] # Extracts the current value of the indicator for the Low
    
    # Defines the value of the signal according to the trading rule
    if (NHR > Threshold_ranges[1] and NLR < Threshold_ranges[0]):
        signal = 1
    elif (NLR > Threshold_ranges[0] and NHR < Threshold_ranges[1]):
        signal = -1
    else:
        signal = 0

    return(signal)

In [44]:
def mapper(df):
    signal = []
    for inst in df['inst'].unique().tolist():
        dfaux = df.loc[df['inst']==inst,:].drop(['hora', 'inst'], axis = 1)
        signal.append((inst, get_signal(dfaux)))
    return [x for x in signal if x[1] != 0]

In [46]:
url = "https://stream-fxpractice.oanda.com/v3/accounts/" + account + "/pricing/stream"
headers = {'Authorization':"Bearer " + token}
params = {"instruments":Tickers_stream}
s = requests.Session()
req = requests.Request("GET",url, headers = headers, params = params)
pre = req.prepare()
resp = s.send(pre, stream = True)
print(resp.reason)
dictf_est = {'inst':dict()}
dicf_bkt = {'inst':dict()}

for i in resp.iter_lines(1):
    
    j = i.decode()
    var = json.loads(j)
    if (var['type'] != 'HEARTBEAT'):
        
        #crea si no existe el instrumento
        if var['instrument'] not in dictf_est['inst'].keys():# or dictf_est['inst'][var['instrument']]=={}:
            
            dictf_est['inst'][var['instrument']] ={0 :{'open':float(var['bids'][0]['price']),
                                                  'high':float(var['bids'][0]['price']),
                                                  'low':float(var['bids'][0]['price']),
                                                  'close':float(var['bids'][0]['price']),
                                                  'hora':var['time']}}

        # Crea si existe el instrumento pero ya no cabe en la vela
        elif str(pd.to_datetime(var['time']) - pd.to_datetime(dictf_est['inst'][var['instrument']][max(dictf_est['inst'][var['instrument']].keys())]['hora'])).split(' ')[2].split(':')[1] != '00':
            
            dictf_est['inst'][var['instrument']][1+max(dictf_est['inst'][var['instrument']].keys())] = {'open':float(var['bids'][0]['price']),
                                                                                              'high':float(var['bids'][0]['price']),
                                                                                              'low':float(var['bids'][0]['price']),
                                                                                              'close':float(var['bids'][0]['price']),
                                                                                              'hora':var['time']}

        # Actualiza si aún cabe en la vela
        else:
            dictf_est['inst'][var['instrument']][max(dictf_est['inst'][var['instrument']].keys())]['close'] = float(var['bids'][0]['price'])

            if float(var['bids'][0]['price']) > dictf_est['inst'][var['instrument']][max(dictf_est['inst'][var['instrument']].keys())]['high']:
                dictf_est['inst'][var['instrument']][max(dictf_est['inst'][var['instrument']].keys())]['high'] = float(var['bids'][0]['price'])
            elif float(var['bids'][0]['price']) < dictf_est['inst'][var['instrument']][max(dictf_est['inst'][var['instrument']].keys())]['low']:
                dictf_est['inst'][var['instrument']][max(dictf_est['inst'][var['instrument']].keys())]['low'] = float(var['bids'][0]['price'])

        if len(dictf_est['inst'][var['instrument']].keys()) == 11:
            df = pd.DataFrame(columns=['open','high','low','close','inst'],index=[1],data=[[np.nan]*5])
            for var1 in dictf_est['inst'].keys():
                tdf = pd.DataFrame.from_dict(dictf_est['inst'][var1],orient='index')
                tdf['inst'] = var1
                df = pd.concat([df,tdf])
            df = df.dropna(how='all',axis=0).reset_index().drop('index',axis=1)

            try:
                for var1 in dictf_est['inst'].keys():
                    del dictf_est['inst'][var1][min(list(dictf_est['inst'][var1].keys()))]
            except:
                del dictf_est['inst'][var1]
            
            signals = mapper(df)
            
            for signal in signals:
                placeOrder(signal[0], units = 10000*signal[1])
        
#         # Para Backtesting
#         if var['instrument'] not in dicf_bkt['inst'].keys():# or dicf_bkt['inst'][var['instrument']]=={}:
#             dicf_bkt['inst'][var['instrument']] ={0 :{'open':float(var['bids'][0]['price']),
#                                                   'high':float(var['bids'][0]['price']),
#                                                   'low':float(var['bids'][0]['price']),
#                                                   'close':float(var['bids'][0]['price']),
#                                                   'hora':var['time']}}
#         # Crea si existe el instrumento pero ya no cabe en la vela
#         elif str(pd.to_datetime(var['time']) - pd.to_datetime(dicf_bkt['inst'][var['instrument']][max(dicf_bkt['inst'][var['instrument']].keys())]['hora'])).split(' ')[2].split(':')[1] != '00':
#             dicf_bkt['inst'][var['instrument']][1+max(dicf_bkt['inst'][var['instrument']].keys())] = {'open':float(var['bids'][0]['price']),
#                                                                                               'high':float(var['bids'][0]['price']),
#                                                                                               'low':float(var['bids'][0]['price']),
#                                                                                               'close':float(var['bids'][0]['price']),
#                                                                                               'hora':var['time']}

#         # Actualiza si aún cabe en la vela
#         else:
#             dicf_bkt['inst'][var['instrument']][max(dicf_bkt['inst'][var['instrument']].keys())]['close'] = float(var['bids'][0]['price'])
            
#             if float(var['bids'][0]['price']) > dicf_bkt['inst'][var['instrument']][max(dicf_bkt['inst'][var['instrument']].keys())]['high']:
#                 dicf_bkt['inst'][var['instrument']][max(dicf_bkt['inst'][var['instrument']].keys())]['high'] = float(var['bids'][0]['price'])
                
#             elif float(var['bids'][0]['price']) < dicf_bkt['inst'][var['instrument']][max(dicf_bkt['inst'][var['instrument']].keys())]['low']:
#                 dicf_bkt['inst'][var['instrument']][max(dicf_bkt['inst'][var['instrument']].keys())]['low'] = float(var['bids'][0]['price'])

#             if pd.datetime.now().time() > pd.to_datetime('2020-04-16 23:59:59').time():
                
#                 df = pd.DataFrame(columns=['open','high','low','close','inst'],index=[1],data=[[np.nan]*5])
#                 for var in dicf_bkt['inst'].keys():
#                     tdf = pd.DataFrame.from_dict(dicf_bkt['inst'][var],orient='index')
#                     tdf['inst'] = var
#                     df = pd.concat([df,tdf])
#                 df = df.dropna(how='all',axis=0).reset_index().drop('index',axis=1)
#                 olddf = pd.read_csv('s3://dataminingclass/info/instrumentos.csv',sep=',')
#                 olddf = olddf.drop('Unnamed: 0',axis=1)
#                 olddf = olddf.rename(columns={'o':'open',
#                                               'h':'high',
#                                               'l':'low',
#                                               'c':'close'})
#                 dfNew = pd.concat([olddf,df])
#                 dfNew.to_csv('instrumentos.csv',index=False,encoding='iso-8859-1',sep=';')
#                 dfNew.to_csv('s3://dataminingclass/info/instrumentos.csv', index=False)
#                 break

OK


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


NameError: name 'singals' is not defined